### Packages

In [1]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras_metrics import KerasMetrics
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import random
import pickle
import decoding2
import decoding_function as df
from Files import nodes
import os
import utility as ut
import livelossplot
sys.path.append('../')

Using TensorFlow backend.


In [35]:
def pipeline(verbs):
    """
    Pipeline receives a list of verbs and predicts a conjugated form for them.
    The prediction is saved in an output file.

    :verbs type: list
    :r type: string
    """
    import coding_function as cf
    from time import gmtime, strftime
    import itertools
    time = strftime("%Y-%m-%d_%H:%M:%S", gmtime())
    name = "output_" + time + ".txt"

    correct = {'#pega#': '#pEgu#', '#sega#': '#sEgu#', '#seka#': '#sEku#',
               '#leva#': '#lEvu#', '#ora#': '#Oru#', '#mora#': '#mOru#',
               '#posta#': '#pOtu#', '#joga#': '#jOgu#', '#sortia#': '#soiu#',
               '#media#': '#meiu#', '#kompo#': '#koiu#', '#po#': '#poiu#', 
               '#menti#': '#mitu#', '#tosi#': '#tusu#', '#kobri#': 
               '#kuro#', '#faze#': '#fasu#', '#mata#': '#matu#', '#paga#': 
               '#pagu#', '#sai#': '#saiu#', '#bate#': '#batu#', '#kome#': 
               '#komu#','#nota#': '#nOta#','#soka#': '#sOku#','#loga#': '#lOga#',
               '#limpa#': '#lipu#','#kasa#': '#kasu#'}

#     f = open('Files/Results/'+ name, "w")
#     f.write("train data set:" + path + '\n' + 'Results:\n' + 'model name: ' + load + '\n')
    test_list = []
    for i in verbs:
        coding = cf.coding(i)
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    sum = 0
    for i,j in list(zip(verbs, prediction)):
#         f.write('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
#                 decoding2.decoding(j) + '\n')
        decoded = decoding2.decoding(j)
        print('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
                decoded + '\n')
        
        if decoded == correct[i]:
            sum = sum + 1    
    print("Accuracy (verbs): %.2f%%" % (sum/len(verbs)*100 ))
    scores = model.evaluate(X, Y)
#     f.write("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[1]*100,
#                                                         model.metrics_names[1], scores[1]*100, 
#                                                         model.metrics_names[2], scores[2]*100,
#                                                         model.metrics_names[3], scores[3]*100 ))
    print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                                   model.metrics_names[1], scores[1]*100, 
                                                                   model.metrics_names[2], scores[2]*100,
                                                                   model.metrics_names[3], scores[3]*100 ))
    
    
    #f.close()
    return #print("\nPredictions saved in the file: " + name) 

### 1. Load Data

In [30]:
#path = '../data/prop_55.csv' #len 464
#path = '../data/prop_65.csv'
#path = '../data/prop_75.csv'
#path = '../data/prop_85.csv'
#path = '../data/prop_95.csv'
path = 'Files/50_50_morar.csv'
X,Y = ut.load_data(path)
batch = len(X)

### 2. Fit Neural Net (Optional)

In [31]:
# 1. Define Model
model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


# 2. Compile model
keras_metrics = KerasMetrics()
model.compile( 
    optimizer='adam', 
    loss='mean_squared_error',
    metrics = [ keras_metrics.fbeta_score,
               keras_metrics.recall,
               keras_metrics.precision]
            )

# 3. Fit model
#stopper = EarlyStopping(monitor='loss', min_delta=0.00005, patience=50, verbose=1, mode='max')
model.fit(X,Y,epochs = 400, batch_size=batch,verbose=False)#,callbacks= [stopper])

# 4. Evaluate model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                               model.metrics_names[1], scores[1]*100, 
                                                               model.metrics_names[2], scores[2]*100,
                                                               model.metrics_names[3], scores[3]*100 ))

# 5. Save model
model.save('../data/test_5050')

 32/192 [====>.........................] - ETA: 0s
loss: 0.73% 
fbeta_score: 99.87% 
recall: 99.75% 
precision: 99.99%


### 2. (Or Load Trained Model)
For example: 

In [4]:
#model = load_model('../data/prop_55',custom_objects={'fbeta_score': keras_metrics.fbeta_score,

#### 2.1 Train existing model

In [ ]:
#model.fit(X,Y,epochs = 150, batch_size=batch,verbose=False,callbacks= [stopper])

### 3. Check Results
Test the prediction for some verbs.

pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#tendi#','#jenti#',
          '#menti#','#hendi#','#tosi#','#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])
    

|       	|       	| front 	| middle  	| back    	|
|:-----:	|-------	|------:	|---------	|---------	|
|       	|       	| v/u   	| v/u     	| v/u     	|
|  **int**  | **stop**  | b/p   	| d/t     	| g/k     	|
|       	| **nasal** | m/-   	| n/-     	| -       	|
| **cont**  | **fric**  | v/f   	| z/s    	| j/x   	|
|       	| **liq**   | l/-   	| r/-     	| -/h     	|
| **vowel** | **high**  | e/i   	| -       	| o/u     	|
|       	| **low**   | -/E   	| -/a     	| -/O     	|

## Results:

## Proporcao 55% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_55    | prop_55       |   209    |  464     |    55%     | 400    |  464         |

In [15]:
path = '../data/prop_55.csv'
X,Y = ut.load_data(path)
keras_metrics = KerasMetrics()
load = '../data/prop_55'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu#, prediction: #pegu#

verb: #sega#, expected: #sEgu#, prediction: #segu#

verb: #seka#, expected: #sEku#, prediction: #seku#

verb: #leva#, expected: #lEvu#, prediction: #levu#

verb: #ora#, expected: #Oru#, prediction: #orru#

verb: #mora#, expected: #mOru#, prediction: #moru#

verb: #posta#, expected: #pOtu#, prediction: #pOtu#

verb: #joga#, expected: #jOgu#, prediction: #jogu#

verb: #sortia#, expected: #soiu#, prediction: #suiu#

verb: #media#, expected: #meiu#, prediction: #meiu#

verb: #kompo#, expected: #koiu#, prediction: #koiu#

verb: #po#, expected: #poiu#, prediction: #poiu#

verb: #menti#, expected: #mitu#, prediction: #mitu#

verb: #tosi#, expected: #tusu#, prediction: #tufu#

verb: #kobri#, expected: #kuro#, prediction: #koru#

verb: #faze#, expected: #fasu#, prediction: #fasu#

verb: #mata#, expected: #matu#, prediction: #matu#

verb: #paga#, expected: #pagu#, prediction: #pagu#

verb: #sai#, expected: #saiu#, prediction: #saiu#

verb: #bat

## Proporcao 65% Irregulares


|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_65     | prop_65       |      209        |  579        |    65%     | 400    |  579         |

In [18]:
path = '../data/prop_65.csv'
X,Y = ut.load_data(path)
keras_metrics = KerasMetrics()
load = '../data/testing'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu#, prediction: #peju#

verb: #sega#, expected: #sEgu#, prediction: #segu#

verb: #seka#, expected: #sEku#, prediction: #seku#

verb: #leva#, expected: #lEvu#, prediction: #levu#

verb: #ora#, expected: #Oru#, prediction: #orru#

verb: #mora#, expected: #mOru#, prediction: #moru#

verb: #posta#, expected: #pOtu#, prediction: #bOtu#

verb: #joga#, expected: #jOgu#, prediction: #jogu#

verb: #sortia#, expected: #soiu#, prediction: #suiu#

verb: #media#, expected: #meiu#, prediction: #meiu#

verb: #kompo#, expected: #koiu#, prediction: #koiu#

verb: #po#, expected: #poiu#, prediction: #poiu#

verb: #menti#, expected: #mitu#, prediction: #mitu#

verb: #tosi#, expected: #tusu#, prediction: #tufu#

verb: #kobri#, expected: #kuro#, prediction: #koru#

verb: #faze#, expected: #fasu#, prediction: #fasu#

verb: #mata#, expected: #matu#, prediction: #matu#

verb: #paga#, expected: #pagu#, prediction: #pagu#

verb: #sai#, expected: #saiu#, prediction: #saiu#

verb: #bat

## Proporcao 75% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_75       | prop_75       |            300          |  400        |    75%     | 400   |  400         |

In [22]:
path = '../data/prop_75.csv'
X,Y = ut.load_data(path)
keras_metrics = KerasMetrics()
load = '../data/prop_75'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu#, prediction: #peku#

verb: #sega#, expected: #sEgu#, prediction: #sigu#

verb: #seka#, expected: #sEku#, prediction: #siku#

verb: #leva#, expected: #lEvu#, prediction: #levu#

verb: #ora#, expected: #Oru#, prediction: #orru#

verb: #mora#, expected: #mOru#, prediction: #muru#

verb: #posta#, expected: #pOtu#, prediction: #pOtu#

verb: #joga#, expected: #jOgu#, prediction: #jogu#

verb: #sortia#, expected: #soiu#, prediction: #suiu#

verb: #media#, expected: #meiu#, prediction: #meiu#

verb: #kompo#, expected: #koiu#, prediction: #koiu#

verb: #po#, expected: #poiu#, prediction: #poiu#

verb: #menti#, expected: #mitu#, prediction: #mitu#

verb: #tosi#, expected: #tusu#, prediction: #tufu#

verb: #kobri#, expected: #kuro#, prediction: #koru#

verb: #faze#, expected: #fasu#, prediction: #fasu#

verb: #mata#, expected: #matu#, prediction: #matu#

verb: #paga#, expected: #pagu#, prediction: #bagu#

verb: #sai#, expected: #saiu#, prediction: #saiu#

verb: #bat

## Proporcao 85% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_85       | prop_85       |             340         |  400        |    85%     | 400   |  400         |

In [26]:
path = '../data/prop_85.csv'
X,Y = ut.load_data(path)
keras_metrics = KerasMetrics()
load = '../data/prop_85'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu#, prediction: #peku#

verb: #sega#, expected: #sEgu#, prediction: #sigu#

verb: #seka#, expected: #sEku#, prediction: #siku#

verb: #leva#, expected: #lEvu#, prediction: #levu#

verb: #ora#, expected: #Oru#, prediction: #orru#

verb: #mora#, expected: #mOru#, prediction: #muru#

verb: #posta#, expected: #pOtu#, prediction: #pOtu#

verb: #joga#, expected: #jOgu#, prediction: #jogu#

verb: #sortia#, expected: #soiu#, prediction: #suiu#

verb: #media#, expected: #meiu#, prediction: #meiu#

verb: #kompo#, expected: #koiu#, prediction: #koiu#

verb: #po#, expected: #poiu#, prediction: #poiu#

verb: #menti#, expected: #mitu#, prediction: #mitu#

verb: #tosi#, expected: #tusu#, prediction: #tupu#

verb: #kobri#, expected: #kuro#, prediction: #koru#

verb: #faze#, expected: #fasu#, prediction: #fasu#

verb: #mata#, expected: #matu#, prediction: #matu#

verb: #paga#, expected: #pagu#, prediction: #pagu#

verb: #sai#, expected: #saiu#, prediction: #saiu#

verb: #bat

## Proporcao 95% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_95       | prop_95       |             380         |  400        |    95%     | 400   |  400         |

In [29]:
keras_metrics = KerasMetrics()
load = '../data/prop_95'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu#, prediction: #bEku#

verb: #sega#, expected: #sEgu#, prediction: #sigu#

verb: #seka#, expected: #sEku#, prediction: #siku#

verb: #leva#, expected: #lEvu#, prediction: #lezu#

verb: #ora#, expected: #Oru#, prediction: #esru#

verb: #mora#, expected: #mOru#, prediction: #miru#

verb: #posta#, expected: #pOtu#, prediction: #pOtu#

verb: #joga#, expected: #jOgu#, prediction: #xugu#

verb: #sortia#, expected: #soiu#, prediction: #suiu#

verb: #media#, expected: #meiu#, prediction: #meiu#

verb: #kompo#, expected: #koiu#, prediction: #koiu#

verb: #po#, expected: #poiu#, prediction: #poiu#

verb: #menti#, expected: #mitu#, prediction: #mitu#

verb: #tosi#, expected: #tusu#, prediction: #tutu#

verb: #kobri#, expected: #kuro#, prediction: #koru#

verb: #faze#, expected: #fasu#, prediction: #fasu#

verb: #mata#, expected: #matu#, prediction: #matu#

verb: #paga#, expected: #pagu#, prediction: #pagu#

verb: #sai#, expected: #saiu#, prediction: #saiu#

verb: #bat

### Experimento 50 - 50

In [36]:
model = load_model('../data/test_5050',custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#nota#','#soka#','#loga#','#bate#','#limpa#','#kasa#'])

verb: #nota#, expected: #nOta#, prediction: #dOtu#

verb: #soka#, expected: #sOku#, prediction: #sOku#

verb: #loga#, expected: #lOga#, prediction: #logu#

verb: #bate#, expected: #batu#, prediction: #batu#

verb: #limpa#, expected: #lipu#, prediction: #lipu#

verb: #kasa#, expected: #kasu#, prediction: #kasu#

Accuracy (verbs): 66.67%
 32/192 [====>.........................] - ETA: 0s
loss: 0.73% 
fbeta_score: 99.87% 
recall: 99.75% 
precision: 99.99%


In [1]:
import livelossplot

Using TensorFlow backend.
